# Behavioral Cloning-Project

## Driving data set

In [1]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
import sklearn
import cv2

from sklearn.model_selection import train_test_split
from keras.layers import Input, Conv2D, Cropping2D, Lambda, Dense, Flatten
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.models import Model

from src import get_driving_data, \
    parse_data_row, \
    steering_image_batch_generator

Using TensorFlow backend.


### Load data

In [2]:
data_dir = os.path.join('/home', 'Simulation_Data')
log_csv = os.path.join(data_dir, 'driving_log.csv')

driving_df = get_driving_data(log_csv)
print(driving_df.loc[5])  # print random example to check that data is as expected

Center image    IMG/center_2018_10_20_21_50_37_206.jpg
Left image        IMG/left_2018_10_20_21_50_37_206.jpg
Right image      IMG/right_2018_10_20_21_50_37_206.jpg
Steering                                     -0.122066
Name: 5, dtype: object


### Exploring data

In [3]:
n_samples = driving_df.shape[0]
mean_steering = driving_df['Steering'].mean()
print(f'number of steering angles recorded: {n_samples} x 3 = {n_samples*3}\nmean steering angle: {mean_steering}')

number of steering angles recorded: 9742 x 3 = 29226
mean steering angle: -0.0011161202688359676


### Create train and validation sets from driving data

In [4]:
# Create training and validation dataset
train_set, validation_set = train_test_split(driving_df.values, test_size=0.2)

train_generator = steering_image_batch_generator(data_dir, train_set)
validation_generator = steering_image_batch_generator(data_dir, validation_set)

# Also create a smaller training and validation dataset for testing models
small_train_set, small_validation_set = train_test_split(driving_df[:1000].values, test_size=0.3)

## PilotNet model

In [5]:
img_height = 160
img_width = 320

input_layer = Input(shape=(img_height, img_width, 3), name='input_image')

# Crop the input image first,
# then normalize it.
x = Cropping2D(cropping=((70, 25), (0, 0)), name='image_cropping')(input_layer)
x = Lambda(lambda n: n / 255.0 - 0.5, name='image_normalization')(x)

x = Conv2D(24, (5, 5), strides=(2, 2), activation='relu', name='conv1')(x)
x = Conv2D(36, (5, 5), strides=(2, 2), activation='relu', name='conv2')(x)
x = Conv2D(48, (5, 5), strides=(2, 2), activation='relu', name='conv3')(x)
x = Conv2D(64, (3, 3), activation='relu', name='conv4')(x)
x = Conv2D(64, (3, 3), activation='relu', name='conv5')(x)
x = Flatten(name='flatten')(x)
x = Dense(100, name='dense1')(x)
x = Dense(50, name='dense2')(x)
x = Dense(10, name='dense3')(x)
x = Dense(1, name='output_angle')(x)

model = Model(input=input_layer, output=x)
model.compile(loss='mse', optimizer='Nadam')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_image (InputLayer)     (None, 160, 320, 3)       0         
_________________________________________________________________
image_cropping (Cropping2D)  (None, 65, 320, 3)        0         
_________________________________________________________________
image_normalization (Lambda) (None, 65, 320, 3)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 31, 158, 24)       1824      
_________________________________________________________________
conv2 (Conv2D)               (None, 14, 77, 36)        21636     
_________________________________________________________________
conv3 (Conv2D)               (None, 5, 37, 48)         43248     
_________________________________________________________________
conv4 (Conv2D)               (None, 3, 35, 64)         27712     
__________

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ou...)`


In [6]:
import time

def get_log_dir(base_path='./logs/PilotNet', path_suffix=None):
    localtime = time.localtime()
    datetime_string = time.strftime("%Y-%m-%d_%H-%M-%S", localtime)
    if path_suffix is not None:
        return f'{base_path}/{path_suffix}/{datetime_string}'
    return f'{base_path}/{datetime_string}'

In [7]:
images, steering = [], []
for row in small_train_set:
    i, s = parse_data_row(data_dir, row)
    images.append(i)
    steering.append(s)
x_train, y_train = np.array(images), np.array(steering)

images, steering = [], []
for row in small_validation_set:
    i, s = parse_data_row(data_dir, row)
    images.append(i)
    steering.append(s)
x_validation, y_validation = np.array(images), np.array(steering)

tensorboard = TensorBoard(log_dir=get_log_dir(path_suffix='/debug'),
                          batch_size=32,
                          write_images=True,
                          write_graph=True,
                          write_grads=True,
                          histogram_freq=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=1, mode='auto')

model.fit(x_train, y_train,
          validation_data=(x_validation, y_validation),
          batch_size=32,
          epochs=10,
          callbacks=[tensorboard, early])

Train on 700 samples, validate on 300 samples
Epoch 1/10
700/700 [==============================] - 2s 4ms/step - loss: 1.0920 - val_loss: 0.0031
Epoch 2/10
700/700 [==============================] - 1s 1ms/step - loss: 0.0027 - val_loss: 0.0016
Epoch 3/10
700/700 [==============================] - 1s 2ms/step - loss: 0.0019 - val_loss: 0.0020
Epoch 4/10
700/700 [==============================] - 1s 1ms/step - loss: 0.0016 - val_loss: 0.0014
Epoch 5/10
700/700 [==============================] - 1s 2ms/step - loss: 0.0011 - val_loss: 0.0013
Epoch 6/10
700/700 [==============================] - 1s 1ms/step - loss: 8.2441e-04 - val_loss: 9.7478e-04
Epoch 7/10
700/700 [==============================] - 1s 2ms/step - loss: 7.5267e-04 - val_loss: 8.3875e-04
Epoch 8/10
700/700 [==============================] - 1s 1ms/step - loss: 6.8166e-04 - val_loss: 6.8388e-04
Epoch 9/10
700/700 [==============================] - 1s 2ms/step - loss: 8.4787e-04 - val_loss: 9.9316e-04
Epoch 10/10
700/700 [=

In [10]:
layer_name = 'image_cropping'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(x_train[None, 0])

In [ ]:
tensorboard = TensorBoard(log_dir=get_log_dir(),
                                  batch_size=self.batch_size,
                                  write_graph=False)
# Save the model according to the conditions
checkpoint = ModelCheckpoint('PilotNet.h5', monitor='val_loss', verbose=1, save_best_only=True,
                             save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=1, mode='auto')

model.fit_generator(train_generator,
                    validation_data=validation_generator,
                    epochs=10,
                    steps_per_epoch=num_train_samples,
                    validation_steps=num_validation_samples,
                    callbacks=[tensorboard, checkpoint, early])

## Xception - transfer learning model

In [ ]:
from keras.applications import Xception
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

img_width = 150
img_height = 150

model = Xception(weights='imagenet',
                 include_top=False,
                 input_shape=(img_height, img_width, 3))

# Freeze all layers of pre-trained model
for layer in model.layers:
    layer.trainable = False

x = model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(1)(x)  # Regression output layer

model_final = Model(input=model.input, output=x)
model_final.compile(loss='mse', optimizer='Nadam')

# Save the model according to the conditions  
checkpoint = ModelCheckpoint('Xception_steering.h5', monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

model_final.fit_generator(train_generator, 
                          samples_per_epoch=len(train_samples), 
                          validation_data=validation_generator,
                          nb_val_samples=len(validation_samples), 
                          nb_epoch=10, 
                          callbacks=[checkpoint, early])